# 1. Selenium 이란?

* 웹브라우저를 자동으로 제어하는 라이브러리
* 본래 다양한 웹브라우저를 사용한 웹 사이트 자동 테스트 도구
* 현재는 웹브라우저를 자동 제어하여 동적 웹사이트에서 정보를 가져오는 용도로도 많이 활용

# 2. 정적 웹 사이트 vs 동적 웹 사이트
* 정적 웹 사이트(Static website): HTML 페이지 안에 웹 브라우저에 표시되는 모든 정보가 포함된 사이트
 * requests 라이브러리를 활용해서 정보 추출 가능
* 동적 웹 사이트(Dynamic website): Java Script를 사용하여 상황에 따라 필요한 정보를 불러와 표시하는 웹 사이트
 * 상황에 맞추어 자바스크립트를 활용해 서버에 정보를 다시 요청하는 형식이라 requests로는 정보 추출 불가능
 * selenium으로 웹브라우저를 조작해 원하는 정보가 다 표시 된 후 정보 추출 필요

# 3. Selenium 웹사이트
[Selenium website](https://www.selenium.dev/documentation/webdriver/)

# 4. Selenium 설치

In [1]:
!pip install selenium
!pip install webdriver-manager
!pip install lxml

# 5. WebDriver Manager 및 WebDriver 설치
* [webdriver manager 설치](https://github.com/SergeyPirogov/webdriver_manager)

In [2]:
import selenium
print(selenium.__version__)

4.12.0


In [3]:
# selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromiumService
# from webdriver_manager.chrome import ChromeDriverManager
# from webdriver_manager.core.utils import ChromeType

# driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import lxml

In [5]:
# 크롬 옵션즈에 User-Agent, lang 같은 정보를 담아 셀레니움을 이용한 크롤링이 아닌 것 처럼 만들기
# options = webdriver.ChromeOptions()
# options.add_experimental_option("excludeSwitches", ["enable-logging"])
# options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')
# options.add_argument("lang=ko_KR")

# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

driver.get("https://search.shopping.naver.com/book/home")  # 크롤링 할 웹사이트 주소

In [6]:
# 네이버 쇼핑에서 오른쪽 끝의 도서검색창을 찾아 검색어 입력
# search_box = driver.find_element(By.XPATH, '//*[@id="lnb"]/div/div[2]/div/div/form/input')
search_box = driver.find_element(By.CSS_SELECTOR, "div._lnbSearch_inner_2EgO1 > form > input")
search_box.send_keys("파이썬"+ Keys.ENTER)

In [7]:
# 검색되어 나온 결과는 스크롤을 아래로 내려야 한 페이지에 40개의 결과가 노출

all_bookList = []
for page in tqdm(range(1,20)):   # 반복문을 이용해 1000px씩 스크롤을 7번 내리도록 해서 전체 페이지 로딩
    y = 0
    y_step = 1000
    for i in range(1,8):
        y = y + y_step
        script = "window.scrollTo({0},{1})".format(0,y)    # 자바스크립트로 위치 지정
        driver.execute_script(script)                      # 스크롤 실행
        time.sleep(1)
    page_html = driver.page_source          # 페이지 끝에서 40개의 책이 로딩된 결과를 저장
    soup = bs(page_html, "html.parser")     # beautifulsoup으로 html을 parsing
    all_bookList.append(soup)               # parsing된 결과를 all_bookList에 저장
    driver.find_element(By.CSS_SELECTOR, "div.Paginator_list_paging__OTDcR > button.Paginator_btn_next__0pdVd").click()  # 다음페이지 넘김 버튼 클릭


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [02:24<00:00,  7.63s/it]


In [8]:
driver.close()

In [9]:
len(all_bookList[0].select_one("#book_list > ul.list_book").select("li"))

40

In [10]:
import pandas as pd

In [11]:
result2 = pd.DataFrame()

# for page in all_bookList:
#     for book in book_list2:
for page in all_bookList:
    for book in tqdm(page.select_one("#book_list > ul.list_book").select("li")):
        title = book.select_one("span.bookListItem_text__bglOw").text
        ranking = book.select_one("div.bookListItem_feature__txTlp").text if book.select_one("div.bookListItem_feature__txTlp") != None else "None"
        author = book.select_one("span.bookListItem_define_data__kKD8t").text if book.select_one("span.bookListItem_define_data__kKD8t") != None else "None"
        pubulisher = book.select_one("div.bookListItem_detail_publish__FgPYQ > span.bookListItem_define_data__kKD8t").text if book.select_one("div.bookListItem_detail_publish__FgPYQ > span.bookListItem_define_data__kKD8t") != None else "None"
        starpoint = book.select_one("div.bookListItem_grade__tywh2").text[2:5] if book.select_one("div.bookListItem_grade__tywh2") != None else "0"
        n_reviews = book.select_one("div.bookListItem_grade__tywh2").text[6:].strip("() ") if book.select_one("div.bookListItem_grade__tywh2") != None else "0"
        price = book.select_one("em").text if book.select_one("em") != None else "0"
        temp = dict(제목 = [title], 랭킹 = [ranking] , 저자 = [author], 출판사 = [pubulisher],
                    평점 = [starpoint], 리뷰수 = [n_reviews], 가격 = [price])
        temp_df = pd.DataFrame(temp)
#         display(temp_df)
        result2 = pd.concat([result2, temp_df])



100%|█████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 506.68it/s]


In [12]:
result2.reset_index(drop=True, inplace=True)

In [13]:
result2

,제목,랭킹,저자,출판사,평점,리뷰수,가격
0,두근두근 파이썬 (만들면서 배우는),컴퓨터/IT 87위,천인국,생능출판,4.8,4,"23,750"
1,"코딩 뇌를 깨우는 파이썬 (문제 해결 능력을 키우는 컴퓨팅 사고부터 알고리즘, 데이...",컴퓨터/IT 97위,존 V. 구태그,한빛미디어,0,0,"31,500"
2,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),컴퓨터/IT 4위,박응용,이지스퍼블리싱,4.9,50,"19,800"
3,파이썬 클린 코드 (유지보수가 쉬운 파이썬 코드를 만드는 비결),컴퓨터/IT 122위,마리아노 아나야,터닝포인트,2.0,1,"26,100"
4,나는 파이썬으로 머신러닝한다 2 (인공지능 자신감 UP 인공지능 약간 매운맛),컴퓨터/IT 113위,"장병철, 이지항 외 2명",씨마스,5.0,1,"20,700"
...,...,...,...,...,...,...,...
755,머신 러닝·딥 러닝에 필요한 기초 수학 with 파이썬,None,조준우,인사이트,4.0,2,"24,300"
756,파이썬과 인공지능 프로젝트 with ASUS Tinker Board (제품 개발 실...,None,"서민우, 전한결",앤써북,0,0,"25,200"
757,파이썬과 코랩기반의 OpenCV로 배우는 영상처리,None,고병철,그린,0,0,"18,000"
758,"파이썬과 리액트를 활용한 주식 자동 거래 시스템 구축 (데이터 수집부터 거래자동화,...",None,박재현,위키북스,5.0,2,"24,200"


In [14]:
finalData = result2[result2["저자"] != 'None']

In [15]:
finalData.drop_duplicates(subset="제목", keep='first')

,제목,랭킹,저자,출판사,평점,리뷰수,가격
0,두근두근 파이썬 (만들면서 배우는),컴퓨터/IT 87위,천인국,생능출판,4.8,4,"23,750"
1,"코딩 뇌를 깨우는 파이썬 (문제 해결 능력을 키우는 컴퓨팅 사고부터 알고리즘, 데이...",컴퓨터/IT 97위,존 V. 구태그,한빛미디어,0,0,"31,500"
2,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),컴퓨터/IT 4위,박응용,이지스퍼블리싱,4.9,50,"19,800"
3,파이썬 클린 코드 (유지보수가 쉬운 파이썬 코드를 만드는 비결),컴퓨터/IT 122위,마리아노 아나야,터닝포인트,2.0,1,"26,100"
4,나는 파이썬으로 머신러닝한다 2 (인공지능 자신감 UP 인공지능 약간 매운맛),컴퓨터/IT 113위,"장병철, 이지항 외 2명",씨마스,5.0,1,"20,700"
...,...,...,...,...,...,...,...
755,머신 러닝·딥 러닝에 필요한 기초 수학 with 파이썬,None,조준우,인사이트,4.0,2,"24,300"
756,파이썬과 인공지능 프로젝트 with ASUS Tinker Board (제품 개발 실...,None,"서민우, 전한결",앤써북,0,0,"25,200"
757,파이썬과 코랩기반의 OpenCV로 배우는 영상처리,None,고병철,그린,0,0,"18,000"
758,"파이썬과 리액트를 활용한 주식 자동 거래 시스템 구축 (데이터 수집부터 거래자동화,...",None,박재현,위키북스,5.0,2,"24,200"


In [16]:
!pip install sqlalchemy

In [17]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/naver_books.db', echo=False)
conn = engine.raw_connection()
finalData.to_sql('naver_books', con=conn, if_exists='append')

C:\Users\hi\AppData\Local\Temp\ipykernel_9552\988611106.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  finalData.to_sql('naver_books', con=conn, if_exists='append')


760

In [18]:
test = pd.read_sql('select * from naver_books', conn)
test

C:\Users\hi\AppData\Local\Temp\ipykernel_9552\2643296496.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test = pd.read_sql('select * from naver_books', conn)


,index,제목,랭킹,저자,출판사,평점,리뷰수,가격
0,0,파이썬과 데이터 과학 (파이썬으로 시작하는 데이터 과학과 기계학습),컴퓨터/IT 34위,"천인국, 박동규",생능출판,5.0,10,"24,700"
1,1,두근두근 파이썬 (만들면서 배우는),컴퓨터/IT 12위,천인국,생능출판,4.7,3,"23,750"
2,2,Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서),컴퓨터/IT 5위,박응용,이지스퍼블리싱,4.8,49,"19,010"
3,3,파이썬으로 만드는 인공지능,컴퓨터/IT 108위,"오일석, 이진선",한빛아카데미,3.0,1,"29,450"
4,4,파이썬 Express,컴퓨터/IT 122위,천인국,생능출판,4.9,12,"28,500"
...,...,...,...,...,...,...,...,...
2275,755,머신 러닝·딥 러닝에 필요한 기초 수학 with 파이썬,None,조준우,인사이트,4.0,2,"24,300"
2276,756,파이썬과 인공지능 프로젝트 with ASUS Tinker Board (제품 개발 실...,None,"서민우, 전한결",앤써북,0,0,"25,200"
2277,757,파이썬과 코랩기반의 OpenCV로 배우는 영상처리,None,고병철,그린,0,0,"18,000"
2278,758,"파이썬과 리액트를 활용한 주식 자동 거래 시스템 구축 (데이터 수집부터 거래자동화,...",None,박재현,위키북스,5.0,2,"24,200"
